In [ ]:
# Code for solving the Roeder problem:
#
#  Find the longest sequence using numbers from 
#  {1,...,100} such that every number is either 
#  a factor or multiple of the previous number. 
#
# https://blog.computationalcomplexity.org/2022/04/the-roeder-problem-was-solved-before-i.html

In [1]:
import networkx as nx

# creates divisor graph on nodes { 1, 2, ..., n } 
#   with two special nodes named 0 and n+1 that
#   are to be the start and end of our path.

def create_divisor_graph(n):
    
    G = nx.Graph()
    G.add_nodes_from(range(n+2))
    
    # add edges from 0 to all other nodes
    for i in range(1,n+1):
        G.add_edge(0,i)
    
    # add edges from all nodes to node n+1
    for i in range(1,n+1):
        G.add_edge(i,n+1)
    
    # add edges between divisors
    for i in range(1,n):
        for m in range(2,n):
            v = i * m
            if v > n:
                break
            G.add_edge(i,v)
            
    return G

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# create a function to separate the subtour elimination constraints
def subtour_elimination(m, where):
    
    # check if LP relaxation at this branch-and-bound node has an integer solution
    if where == GRB.Callback.MIPSOL: 
        
        # retrieve the LP solution
        xval = m.cbGetSolution(m._x)
        
        # which edges are selected?
        edges = [ e for e in m._G.edges if xval[e] > 0.5 ]
        
        # for each subtour, add a constraint
        for component in nx.connected_components( m._G.edge_subgraph( edges ) ):
            
            if m._s not in component:
            
                inner_edges = [ (i,j) for (i,j) in m._G.edges if i in component and j in component ]
                m.cbLazy( gp.quicksum( m._x[e] for e in inner_edges ) <= len(component) - 1 )    

In [4]:
def solve_Roeder_problem(n):

    G = create_divisor_graph(n)
    
    s = 0
    t = n+1

    m = gp.Model()
    x = m.addVars( G.edges, vtype=GRB.BINARY )
    y = m.addVars( G.nodes, vtype=GRB.BINARY )

    y[s].ub = 0
    y[t].ub = 0

    # maximize number of nodes in the s,t-path (excludes s and t)
    m.setObjective( gp.quicksum( y[i] for i in G.nodes ), GRB.MAXIMIZE )

    # one edge should touch node s (and node t)
    m.addConstr( gp.quicksum( x[e] for e in G.edges if e in G.edges(s) ) == 1 )
    m.addConstr( gp.quicksum( x[e] for e in G.edges if e in G.edges(t) ) == 1 )

    # all other nodes should have degree 2
    m.addConstrs( gp.quicksum( x[e] for e in G.edges if e in G.edges(i) ) == 2 * y[i] for i in G.nodes if i not in {s,t} )

    # tell Gurobi that we will be adding (lazy) constraints
    m.Params.lazyConstraints = 1

    # designate the callback routine to be subtour_elimination()
    m._callback = subtour_elimination

    # add the variables and graph to our model object, for use in the callback
    m._x = x
    m._G = G
    m._s = s

    # solve the MIP with our callback
    m.optimize(m._callback)
    
    edges = [ (i,j) for (i,j) in G.edges if x[i,j].x > 0.5 ]
    path = nx.shortest_path(G.edge_subgraph(edges), source=s, target=t)
    print(path[1:-1])  # print the path, except for s and t
    return     

In [5]:
solve_Roeder_problem(n=10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-01
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 48 columns and 82 nonzeros
Model fingerprint: 0x1635d53b
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 12 rows, 46 columns, 82 nonzeros
Variable types: 0 continuous, 46 integer (46 binary)

Root relaxation: objective 9.000000e+00, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [6]:
solve_Roeder_problem(n=100)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 102 rows, 683 columns and 1262 nonzeros
Model fingerprint: 0xf3b5c884
Variable types: 0 continuous, 683 integer (683 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 102 rows, 681 columns, 1262 nonzeros
Variable types: 0 continuous, 681 integer (681 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 8.150000e+01, 184 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   81.50000    0   14    1.00000   81.50000  8050%     -    

In [7]:
solve_Roeder_problem(n=200)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 202 rows, 1499 columns and 2794 nonzeros
Model fingerprint: 0xbc942751
Variable types: 0 continuous, 1499 integer (1499 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 202 rows, 1497 columns, 2794 nonzeros
Variable types: 0 continuous, 1497 integer (1497 binary)

Root relaxation: objective 1.625000e+02, 403 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.50000    0   24    1.00000  162.50000      -    

In [8]:
solve_Roeder_problem(n=300)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 302 rows, 2368 columns and 4432 nonzeros
Model fingerprint: 0xd795dff0
Variable types: 0 continuous, 2368 integer (2368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 302 rows, 2366 columns, 4432 nonzeros
Variable types: 0 continuous, 2366 integer (2366 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 2.425000e+02, 567 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  242.50000    0   26    1.00000  242.50000      -    

In [9]:
solve_Roeder_problem(n=400)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 402 rows, 3269 columns and 6134 nonzeros
Model fingerprint: 0x4a5c40e8
Variable types: 0 continuous, 3269 integer (3269 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 402 rows, 3267 columns, 6134 nonzeros
Variable types: 0 continuous, 3267 integer (3267 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 3.225000e+02, 752 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  322.50000    0   42    1.00000  322.50000      -    

In [10]:
solve_Roeder_problem(n=500)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 502 rows, 4191 columns and 7878 nonzeros
Model fingerprint: 0x85715bd5
Variable types: 0 continuous, 4191 integer (4191 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 502 rows, 4189 columns, 7878 nonzeros
Variable types: 0 continuous, 4189 integer (4189 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 4.030000e+02, 901 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  403.00000    0   42    1.00000  403.00000      -    

In [11]:
solve_Roeder_problem(n=600)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 602 rows, 5145 columns and 9686 nonzeros
Model fingerprint: 0x83ce66d8
Variable types: 0 continuous, 5145 integer (5145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 602 rows, 5143 columns, 9686 nonzeros
Variable types: 0 continuous, 5143 integer (5143 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 4.820000e+02, 1108 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  482.00000    0   56    1.00000  482.00000      -   